## Lang Chain 
Esta seção mostra como usar embeddings para busca semântica e responder a perguntas sobre bases de dados.
Ela usa a biblioteca LangChain.

In [1]:
from dotenv import load_dotenv
load_dotenv()
import re
import requests
import sys
import os
from dotenv import load_dotenv
load_dotenv()


API_KEY = os.getenv("AZURE_OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"

RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
COMPLETIONS_MODEL = os.getenv('DEPLOYMENT_NAME')

os.environ["AZURE_OPENAI_API_KEY"] = API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = RESOURCE_ENDPOINT


In [2]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import AzureOpenAI
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

### Indexando documentos em uma base vetorial


In [5]:
from langchain.document_loaders import TextLoader
loader = TextLoader("./data/constituicao.txt", encoding="latin1")
documents = loader.load()

print(len(documents))

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

print(len(texts))


embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_version="2023-05-15",
    chunk_size=1
)
docsearch = FAISS.from_documents(texts, embeddings)
print('done')

Created a chunk of size 1234, which is longer than the specified 1000
Created a chunk of size 10119, which is longer than the specified 1000
Created a chunk of size 1038, which is longer than the specified 1000
Created a chunk of size 1095, which is longer than the specified 1000
Created a chunk of size 10119, which is longer than the specified 1000


1
791


KeyboardInterrupt: 

### Buscando no documento


In [7]:
from langchain.globals import set_verbose

set_verbose(True)

llm = AzureOpenAI(temperature=0, deployment_name="gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct", api_version='2023-05-15')

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

template = """Question: {question}

"""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=llm)




query = "Qual o percentual do fundo partidário que deve ser destinado a campanhas eleitorais de negros?"
print(llm_chain.run(query))

qa.run(query)

c:\dev\openai\AzureOpenAITutorial\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\dev\openai\AzureOpenAITutorial\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Question: Qual o percentual do fundo partidário que deve ser destinado a campanhas eleitorais de negros?



> Finished chain.
Não há um percentual específico do fundo partidário que deve ser destinado a campanhas eleitorais de negros. No entanto, a Lei nº 9.504/1997, que estabelece normas para as eleições, determina que os partidos políticos devem destinar pelo menos 30% dos recursos do fundo partidário para a promoção da participação feminina na política. Além disso, a Lei nº 13.487/2017, que instituiu o Fundo Especial de Financiamento de Campanha (FEFC), estabelece que pelo menos 30% dos recursos devem ser destinados às candidaturas de mulheres. Portanto, é possível que parte desses recursos seja destinada a campanhas de candidatos negros, mas não há uma porcentagem específica para isso.


> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatt

' 30% (trinta por cento)'

### Salvando o arquivo e carregando de novo


In [16]:
docsearch.save_local('./data/constituicao/')


loaded = FAISS.load_local('./data/constituicao/', embeddings, allow_dangerous_deserialization=True)
